# Demo 3: Properties of the Response Distribution

In Demo 3, we will be focusing on understanding the Response Distribution and its relationship with the Posterior Distribution.
As mentioned in lecture, this is not very intuitive! But with visualization, we believe that it can be made clear.
The demo will be split into 2 parts.
Each demo will be accompanied by **guiding questions**, which will help with the learning process as we go.
    
In **Part A**, we will explore how the perceived hand location (i.e., posterior mean estimate) varies across many trials (i.e., their distribution).

In **Part B**, we will explore the differences between the properties of the *Posterior Distribution* and the *Response Distribution*.


## Demo 3A: Hand localization across trials

In the current demo, we will explore the factors contributing to how the perceived hand location (i.e., maximum a posteriori) varies across many trials.
The below plot illustrates twenty different trials along with their inference process (i.e., belief distributions). The red curves corresponds to each trials' proprioceptive likelihood.
The blue curve corresponds to the prior for hand location. The purple curve corresponds to a single illustrative Posterior distribution.
And finally, the purple dashed lines correspond to where the hand was perceived on every trial. 

As before, you can adjust both the measurement (i.e., stimulus location, and noise) and the prior distribution (i.e., mean and uncertainty).
    
### Guiding Questions
1. To see what factors affect behavioral variability, adjust the means and variances of the measurement and the prior distribution. 
What is the contribution of the likelihood distributions? What is the contribution of the prior distributions?

2. Visually compare the spread of perceived hand locations and the width of the illustrative posterior. What do you notice?
Hint: Consider using extreme values for each parameter.


In [3]:

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, Checkbox, VBox, HBox
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# ---------------------------------------------------------------------------
# Global axis limits (stay *constant* no matter what the sliders do)
P_RANGE = (-10, 10)
SIGMA_P_RANGE = (1, 5)

PR_RANGE = (-10, 10)          # from sliders
SIGMA_PR_RANGE = (1, 10)    # from sliders

XLIM = (-20, 20)   # (-17, 17)
YLIM = (-0.002, 0.05)         # room for baseline + PDF peak (0.15)


def bayes_acrossTrials(s_p: float = 0.0, sigma_p: float = 1.0,
                      mu_pr: float = 0.0, sigma_pr: float = 1.0):
    

    fig, ax = plt.subplots(figsize=(8, 6))

    # Measurement
    xs = np.linspace(XLIM[0], XLIM[1], 400)
    rng = np.random.default_rng()

    # Prior Distribution
    var_pr = sigma_pr**2
    norm_pr = ((1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr))).sum()
    pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr) )
    pdf_pr_norm = pdf_pr / norm_pr
    ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)
    n_samples = 20
    for n in range(int(n_samples)):
        x_obs_p = rng.normal(loc=s_p, scale=sigma_p, size=1)
        # Proprioceptive Likelihood
        var_p = sigma_p**2
        norm_p = ((1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((0 - xs)** 2 / var_p))).sum()
        pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((x_obs_p - xs)** 2 / var_p) )
        pdf_p_norm = pdf_p / norm_p
        ax.plot(xs, pdf_p_norm, color="red", linewidth=2)
        w_p = var_p**-1/(var_p**-1 + var_pr**-1)
        w_pr = 1-w_p

    
        mu_post = w_p*x_obs_p + w_pr* mu_pr
        ax.axvline(mu_post, color="purple", linestyle="--")
        

    # Proprioceptive Likelihood
    var_p = sigma_p**2
    norm_p = ((1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((0 - xs)** 2 / var_p))).sum()
    pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((s_p - xs)** 2 / var_p) )
    pdf_p_norm = pdf_p / norm_p


    pdf_post = pdf_p*pdf_pr
    pdf_post_norm = pdf_post / pdf_post.sum() # Scaling the pdf for better visual
    ax.plot(xs, pdf_post_norm, color="purple", linewidth=2)

    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45*XLIM[1],0.9*YLIM[1],'Likelihood Dist.',color="red",fontsize=12)
    ax.text(0.45*XLIM[1],0.83*YLIM[1],'Prior Dist.',color="blue",fontsize=12)
    ax.text(0.45*XLIM[1],0.76*YLIM[1],'Posterior(s)',color="purple",fontsize=12)

    # Aesthetics
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()

# ---------------------------------------------------------------------------
# controls
controls = {
    "s_p": FloatSlider(value=-7.0, min=P_RANGE[0], max=P_RANGE[1], step=0.1,
                       description="P Stimulus:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_p": FloatSlider(value=2.0, min=SIGMA_P_RANGE[0], max=SIGMA_P_RANGE[1], step=0.1,
                          description="P Noise σ:", continuous_update=True),
    "mu_pr": FloatSlider(value=5.0, min=PR_RANGE[0], max=PR_RANGE[1], step=0.1,
                       description="Prior:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_pr": FloatSlider(value=3, min=SIGMA_PR_RANGE[0], max=SIGMA_PR_RANGE[1], step=0.1,
                          description="Prior σ:", continuous_update=True),

}

interactive_plot = interactive(bayes_acrossTrials, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["s_p"], controls["sigma_p"],controls["mu_pr"], controls["sigma_pr"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()

## Add box to show the clicks

## Demo 3B: Response vs. Posterior Distrubition

It is a common mistake in modelling to assume that the response and posterior distributions are equivalent. 
In the present demo, we will more directly explore how the response distribution differs from the posterior distribution. 
The below plot illustrates the three *subjective* distributions: 
A likelihood distribution (red curve), a prior distribution (blue curve), and a posterior distribution (purple curve). 
Whereas these distributions fluctuate across trials (due to noise), for illustrative purposes we present them as stable. 
Accompanying these distributions is the response distribution (green curve), 
which is an *objective* distribution over the behavioral responses across trials. 

With this plot you can draw a more direct comparison between the posterior and the response distributions, 
and appreciate better how they differ and why.

As before, you can adjust both the measurement (i.e., stimulus location, and noise) and the prior distribution (i.e., mean and uncertainty).

    
### Guiding Questions
1. Keeping all other parameters at their default values,
what happens to the response distribution when you increase and decrease the prior uncertainty (Prior σ)?

2. Repeat the same steps as in question 1, but by changing the measurement noise. 
What combinations of "P Noise σ" and "Prior σ" lead to the greatest differences between the posterior and response distributions?
                               
3. Why do the posterior and response distributions (in the current scenario) always differ?

In [4]:


# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120



def response_dist(s_p: float = 0.0, sigma_p: float = 1.0,
                  mu_pr: float = 0.0, sigma_pr: float = 1.0):
    

    fig, ax = plt.subplots(figsize=(8, 6))

    # Measurement
    xs = np.linspace(XLIM[0], XLIM[1], 400)
    x_obs_p = s_p # for this demo, we will not include noise
    


    # Proprioceptive Likelihood
    var_p = sigma_p**2
    norm_p = ((1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((0 - xs)** 2 / var_p))).sum()
    pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((x_obs_p - xs)** 2 / var_p) )
    pdf_p_norm = pdf_p / norm_p
    ax.plot(xs, pdf_p_norm, color="red", linewidth=2)



    # Prior Distribution
    var_pr = sigma_pr**2
    norm_pr = ((1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr))).sum()
    pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr) )
    pdf_pr_norm = pdf_pr / norm_pr
    ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)


    # Posterior Distribution

    pdf_post = pdf_p*pdf_pr
    pdf_post_norm = pdf_post / pdf_post.sum() # Scaling the pdf for better visual
    ax.plot(xs, pdf_post_norm, color="purple", linewidth=2)

    # Response Distribution
 
    # Calculate mean and variance of the rd
    w_p = var_p**-1/(var_p**-1 + var_pr**-1)
    w_pr = 1-w_p
    mu_rd = w_p*x_obs_p + w_pr*mu_pr
    var_rd = w_p**2*var_p

    
    pdf_rd = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - mu_rd)** 2 / var_rd) )
    pdf_rd_norm = pdf_rd / pdf_rd.sum() # Scaling the pdf for better visual
    ax.plot(xs, pdf_rd_norm, color="green", linewidth=2)

    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45*XLIM[1],0.9*YLIM[1],'Likelihood Dist.',color="red",fontsize=12)
    ax.text(0.45*XLIM[1],0.83*YLIM[1],'Prior Dist.',color="blue",fontsize=12)
    ax.text(0.45*XLIM[1],0.76*YLIM[1],'Posterior Dist.',color="purple",fontsize=12)
    ax.text(0.45*XLIM[1],0.69*YLIM[1],'Response Dist.',color="green",fontsize=12)

    # Aesthetics
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "s_p": FloatSlider(value=-7.0, min=P_RANGE[0], max=P_RANGE[1], step=0.1,
                       description="P Stimulus:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_p": FloatSlider(value=2.0, min=SIGMA_P_RANGE[0], max=SIGMA_P_RANGE[1], step=0.1,
                          description="P Noise σ:", continuous_update=True),
    "mu_pr": FloatSlider(value=5.0, min=PR_RANGE[0], max=PR_RANGE[1], step=0.1,
                       description="Prior:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_pr": FloatSlider(value=3, min=SIGMA_PR_RANGE[0], max=SIGMA_PR_RANGE[1], step=0.1,
                          description="Prior σ:", continuous_update=True),

}

interactive_plot = interactive(response_dist, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["s_p"], controls["sigma_p"],controls["mu_pr"], controls["sigma_pr"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()
 